In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd
import requests
import snowflake

import sqlalchemy
from sqlalchemy import create_engine

import datetime
from datetime import date
from datetime import timedelta

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# use your own credentials
username='username'
useremail='email'
password='SBDpassword'

## Below is the code to link SnowFlake Database, do not change

In [ ]:
def get_oauth_token(svc_username=None, svc_password=None):
    """
    Retrives the authentication token for SBD Snowflake.
    This function automatically requests for username and password from the user through interactive prompts.
    If using service account credentials, they can be passed as svc_username and svc_password.
    Keywords:
    svc_username -- Service account username for which the token should be provided.
    svc_password -- Password corresponding to the service account user.
    Example usage:
    # Authentication with your personal user id.
    from sbd_common_utils.snowflake_utils import get_oauth_token
    access_token = get_oauth_token()
    # Authentication with service account.
    from sbd_common_utils.snowflake_utils import get_oauth_token
    from sbd_common_utils.common_utils import get_service_account_creds
    username, password = get_service_account_creds("/datascience/sandbox/someapp/service-account")
    access_token = get_oauth_token(username, password)
    """

    if svc_username and svc_password:
        username = svc_username
        password = svc_password
    else:
        print("Username and password isn't provided. Requesting user...")
        username = os.environ["service_account_id"]
        password = get_password_util(
                pass_value=os.environ.get("service_account_password", None),
                pass_name="Please Enter Service Account Password:  ",
            )


    r = requests.post(
        "https://ssoprod.sbdinc.com/as/token.oauth2",
        data={
            "client_id": "Snowflake",
            "grant_type": "password",
            "username": username,
            "password": password,
            "client_secret": 'f9sq630wmLP6UjpSsOk7kTuP6xccCrSOC4YhE1VdTq3GCupqR7gjYcpuhEGRJ9e0',
            "scope": "session:role-any",
        },
    )
    r.raise_for_status()
    access_token = r.json()["access_token"]
    return access_token


In [ ]:
def getsnowflakecursor(snowflakedatabase,
                       snowflakewarehouse,
                       snowflakeschema,
                       snowflakerole,
                       username,
                       userpassword
                      ):
    import requests
    import snowflake.connector
    
    

    access_token = get_oauth_token(svc_username=username,
                               svc_password=userpassword)

    # Gets the version
    activesnowflakeconnector = snowflake.connector.connect(account='sbd_caspian.us-east-1', 
                                                           authenticator='oauth', 
                                                           token=access_token,
                                                          warehouse=snowflakewarehouse,
                                                          database=snowflakedatabase,
                                                          role=snowflakerole,
                                                          schema=snowflakeschema)
    activesnowflakecursor = activesnowflakeconnector.cursor()
    


    sqlalchemyengine = create_engine(f"snowflake://sbd_caspian.us-east-1.snowflakecomputing.com", creator=lambda: activesnowflakeconnector)
    

    return(activesnowflakeconnector,activesnowflakecursor,sqlalchemyengine)


In [ ]:
def get_spark_configs(snowflakedatabase,
                       snowflakewarehouse,
                       snowflakeschema,
                       snowflakerole,
                       username,
                       userpassword,
                     useremail):
    
    
    spark_snowflake_configs = dict(
                sfUrl = 'sbd_caspian.us-east-1.snowflakecomputing.com',
                sfUser = useremail,  
                sfAuthenticator = "oauth",
                sfRole = snowflakerole,
                sfDatabase = snowflakedatabase,
                sfSchema = snowflakeschema,
                sfWarehouse = snowflakewarehouse,  
                sfToken=get_oauth_token(username,userpassword)
                )
    return(spark_snowflake_configs)

In [ ]:
get_oauth_token(username, password)

## You can modify the cell below (dabase, role, schemas) before reading tables

In [ ]:
activesnowflakeconnector,activesnowflakecursor,sqlalchemyengine = getsnowflakecursor(snowflakedatabase='PROD_EDW',
                                                                                     snowflakewarehouse='DEV_AIDA_WH',
                                                                                     snowflakeschema = 'DIMENSIONS',
                                                                                     snowflakerole = 'OPERATIONS_CREATOR_RO',
                                                                                     username = username,
                                                                                     userpassword = password
                                                                                    )

In [ ]:
# this is to pick up the useful column from the product cost table for SAPC11
# as the cost structure is different than SAPE03
# it is not efficient to read the entire table, suggested to pick the product keys in scope in SQL query directly before further analysis
# instead of reading entire table and select the product key later, the code may crash 

querypcc11 = """
select src_sys_key
, LOC_KEY
, COST_EFF_FROM_DTE
, prod_key
, prod_typ_lkey
, prod_cost_lot_sz
, lcrncy_key
, full_cost
, fixed_cost
, FULLCOST_1
, FIXEDCOST_1
, FULLCOST_2
, FIXEDCOST_2
, FULLCOST_3
, FIXEDCOST_3
, FULLCOST_4
, FIXEDCOST_4
, FULLCOST_5
, FIXEDCOST_5
, FULLCOST_6
, FIXEDCOST_6
, FULLCOST_7
, FIXEDCOST_7
, FULLCOST_8
, FIXEDCOST_8
, FULLCOST_9
, FIXEDCOST_9
, FULLCOST_10
, FIXEDCOST_10
, FULLCOST_11
, FIXEDCOST_11
, FULLCOST_12
, FIXEDCOST_12
, FULLCOST_13
, FIXEDCOST_13
, FULLCOST_14
, FIXEDCOST_14
, FULLCOST_15
, FIXEDCOST_15
, FULLCOST_16
, FIXEDCOST_16
, FULLCOST_17
, FIXEDCOST_17
, FULLCOST_18
, FIXEDCOST_18
from prod_edw.dimensions.dim_product_cost
where src_sys_key = 'SAPC11' and prod_key = 'N408124'

"""


In [ ]:
# this is to pick up the useful column from the product cost table for SAPE03
# as the cost structure is different than SAPE03
# it is not efficient to read the entire table, suggested to pick the product keys in scope in SQL query directly before further analysis
# instead of reading entire table and select the product key later, the code may crash 

querypce03 = """
select src_sys_key
, LOC_KEY
, COST_EFF_FROM_DTE
, prod_key
, prod_typ_lkey
, prod_cost_lot_sz
, lcrncy_key
, full_cost
, fixed_cost
, FULLCOST_11
, FIXEDCOST_11
, FULLCOST_12
, FIXEDCOST_12
, FULLCOST_13
, FIXEDCOST_13
, FULLCOST_20
, FIXEDCOST_20
, FULLCOST_30
, FIXEDCOST_30
, FULLCOST_40
, FIXEDCOST_40
, FULLCOST_50
, FIXEDCOST_50
, FULLCOST_60
, FIXEDCOST_60
, FULLCOST_65
, FIXEDCOST_65
, FULLCOST_70
, FIXEDCOST_70
, FULLCOST_80
, FIXEDCOST_80
, FULLCOST_90
, FIXEDCOST_90
, FULLCOST_95
, FIXEDCOST_95
, FULLCOST_99
, FIXEDCOST_99
from prod_edw.dimensions.dim_product_cost
where src_sys_key = 'SAPE03' and prod_key = 'LXUB19610A'
"""


In [ ]:
dfpc1 = pd.read_sql(querypcc11,sqlalchemyengine)
dfpc1.head()

In [ ]:
# Only keep the most recent cost (based on cost_eff_from_date column)
dfpc1a = dfpc1.sort_values(by = ['prod_key', 'loc_key', 'cost_eff_from_dte'], ascending = [True, True, False]).drop_duplicates(['prod_key', 'loc_key'])
dfpc1a.head()

In [ ]:
dfpc2 = pd.read_sql(querypce03,sqlalchemyengine)
dfpc2.head()

In [ ]:
# Only keep the most recent cost (based on cost_eff_from_date column)
dfpc2a = dfpc2.sort_values(by = ['prod_key', 'loc_key', 'cost_eff_from_dte'], ascending = [True, True, False]).drop_duplicates(['prod_key', 'loc_key'])
dfpc2a.head()

In [ ]:
# modify the e03 table column header to replace the cost component number

e03 = [['10', 'Material'], ['11', 'Freight'], ['12', 'Duty'],['13', 'ReadyToSell'], ['20','Subcon.'], 
        ['30', 'Consumbales'], ['40','Std.Adj1'], ['50', 'Std.Adj2'], ['60', 'Labor'], ['65', 'SetUpCost'],
        ['70', 'Overhead'], ['80', 'Tooling'], ['90', 'OtherOH'],['95', 'Std.Adj3'], ['99', 'B&DKitting']]
e03headerfull = pd.DataFrame(data = e03, columns = ['CostComp', 'CostCompName'])
e03headerfull['CostType'] = 'fullcost_'

e03headerfixed = pd.DataFrame(data = e03, columns = ['CostComp', 'CostCompName'])
e03headerfixed['CostType'] = 'fixedcost_'

e03header = pd.concat([e03headerfull, e03headerfixed], axis = 0).reset_index(drop = True)

e03header['Original'] = e03header['CostType']+e03header['CostComp'] 
e03header['New'] = e03header['CostType']+e03header['CostCompName'] 


e03header

In [ ]:
# modify the C11 table column header to replace the cost component number

c11 = [['1', 'Material'], ['2', 'InterplantComp'], ['3', 'FreightAbsoption'],['4', 'MaterialAdj1'], ['5','MaterialAdj2'], 
        ['6', 'Labor'], ['7','LaborAdj'], ['8', 'Overhead'], ['9', 'PurchasingOH'], ['10', 'BudCreditTooling'],
        ['11', 'MiscSupplies'], ['12', 'OHAdj'], ['13', 'MaterialWaste'],['14', 'CashDiscounts'], ['15', 'SubContract'],
        ['16','ExternalProcess'],['17','FixedOHSupport'], ['18','Duty']]
c11headerfull = pd.DataFrame(data = c11, columns = ['CostComp', 'CostCompName'])
c11headerfull['CostType'] = 'fullcost_'

c11headerfixed = pd.DataFrame(data = c11, columns = ['CostComp', 'CostCompName'])
c11headerfixed['CostType'] = 'fixedcost_'

c11header = pd.concat([c11headerfull, c11headerfixed], axis = 0).reset_index(drop = True)

c11header['Original'] = c11header['CostType']+c11header['CostComp'] 
c11header['New'] = c11header['CostType']+c11header['CostCompName'] 


c11header

In [ ]:
column_dicte03 = dict(zip(e03header['Original'], e03header['New']))
dfpc2a.rename(columns=column_dicte03, inplace=True)

In [ ]:
column_dictc11 = dict(zip(c11header['Original'], c11header['New']))
dfpc1a.rename(columns=column_dictc11, inplace=True)
dfpc1a.head()

In [ ]:
# add the location information (need to update the link based on user's path)

pathloc = 'C:\\Users\\HXW0316\\Stanley Black & Decker\\Supply Chain Development - General\\Projects\\Data Collected\\SF data and sample code\\'

dfloc = pd.read_csv(pathloc+'20230613_SFMaster_Location.csv')
dfloc2 = dfloc[['src_sys_key', 'plant_no', 'loc_name', 'match_cd_name', 'city_name', 'cntry_key']].reset_index(drop = True)
dfloc2.head(2)

In [ ]:
dfpc1b = pd.merge(dfpc1a, dfloc2, how = 'left', left_on = ['src_sys_key', 'loc_key'], right_on = ['src_sys_key', 'plant_no'], suffixes=('', '_drop')).reset_index(drop = True)
dfpc1b.drop([col for col in dfpc2b.columns if 'drop' in col], axis=1, inplace=True)

dfpc1b

In [ ]:
dfpc2b = pd.merge(dfpc2a, dfloc2, how = 'left', left_on = ['src_sys_key', 'loc_key'], right_on = ['src_sys_key', 'plant_no'], suffixes=('', '_drop')).reset_index(drop = True)
dfpc2b.drop([col for col in dfpc2b.columns if 'drop' in col], axis=1, inplace=True)

dfpc2b